In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time

In [9]:
df1 = pd.read_csv("../csv/meneame_scraped_final_2.csv", encoding="utf-8")
df2 = pd.read_csv("../csv/meneame_scraped_final.csv", encoding="utf-8")

In [17]:
df = pd.concat([df1,df2], ignore_index=True)


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287091 entries, 0 to 287090
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   news_id          287091 non-null  int64         
 1   title            287091 non-null  object        
 2   content          287074 non-null  object        
 3   meneos           287091 non-null  int64         
 4   clicks           287091 non-null  int64         
 5   karma            287091 non-null  int64         
 6   positive_votes   287091 non-null  int64         
 7   anonymous_votes  287091 non-null  int64         
 8   negative_votes   287091 non-null  int64         
 9   category         287091 non-null  object        
 10  comments         287091 non-null  int64         
 11  published_date   287091 non-null  datetime64[ns]
 12  author           287091 non-null  object        
 13  source           287091 non-null  object        
 14  source_link      287

In [27]:
df["published_date"] = pd.to_datetime(df["published_date"])
df["scraped_date"] = pd.to_datetime(df["scraped_date"])

In [31]:
df = df.drop_duplicates(subset=["news_id"])

# Procesamiento de 'Provincia'


In [34]:
#extracion de provincia desde la columna 'title' para obtener mas informacion en la columna 'provincia
#porque tenemos muchas instancias 'desconocidas'

spanish_provinces = [
    "madrid", "barcelona", "valencia", "sevilla", "zaragoza", "malaga", "murcia", "palma", "las-palmas",
    "bilbao", "alicante", "cordoba", "valladolid", "vigo", "gijon", "hospitalet", "a-coruna", "granada", 
    "vitoria", "elche", "santa-cruz-de-tenerife", "oviedo", "badalona", "cartagena", "terrassa", 
    "jerez-de-la-frontera", "sabadell", "mostoles", "salamanca", "santander", "leganes", "almeria", 
    "burgos", "albacete", "getafe", "san-sebastian", "cadiz", "huelva", "logrono", "badajoz", "tarragona",
    "lleida", "jaen", "ourense", "gerona", "lugo", "santiago", "ponferrada", "palencia", "melilla", "ceuta"
]

def extract_province(texto):
    for province in spanish_provinces:
        if province in texto.lower():
            return province  
    return "Desconocido"

df["province_from_title"] = df["title"].apply(extract_province)

#aqui hice una funcion que hace merge de las tres columnas 

def merge_cols(row):
    for col in ["provincia", "province_from_title"]:
        if row[col] != "Desconocido":
            return row[col].lower()
    return None

df["provincia"] = df.apply(merge_cols, axis=1)
df = df.drop(columns=["province_from_title"])

/var/folders/x1/dfk_lr0j0pg5th31583lwbxc0000gn/T/ipykernel_88843/1286549722.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["province_from_title"] = df["title"].apply(extract_province)
/var/folders/x1/dfk_lr0j0pg5th31583lwbxc0000gn/T/ipykernel_88843/1286549722.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["provincia"] = df.apply(merge_cols, axis=1)


# Procesamiento de 'comunidad'

In [38]:
def comunidad_proc(row):
    if row["comunidad"] == "Desconocido":
        row["comunidad"] = None
    return row
df = df.apply(comunidad_proc, axis=1)

# Procesamiento de Categorias


In [42]:
news_category_map = {
    "politica": "Politica y Sociedad",
    "corruPPción": "Politica y Sociedad",
    "EleccionesE": "Politica y Sociedad",
    "Podemos": "Politica y Sociedad",
    "República": "Politica y Sociedad",
    "RentaBásica": "Politica y Sociedad",
    "Cayetanos": "Politica y Sociedad",
    "corruptcion": "Politica y Sociedad",
    "LuchaDClases": "Politica y Sociedad",
    "Anarquismo": "Politica y Sociedad",
    "DDHH": "Politica y Sociedad",
    "Autonomos": "Politica y Sociedad",
    "Justicia7291": "Politica y Sociedad",
    "Tribunales": "Politica y Sociedad",
    "actualidad": "Politica y Sociedad",
    "ZasSmeame": "Politica y Sociedad",
    "Campañas": "Politica y Sociedad",
    "corrupcion": "Politica y Sociedad",
    "Fachéame": "Politica y Sociedad",
    "VIOLENCIA_G": "Politica y Sociedad",
    "prensa": "Politica y Sociedad",

    
    "tecnología": "Tecnologia y Ciencia",
    "Informática": "Tecnologia y Ciencia",
    "GNU_LINUX": "Tecnologia y Ciencia",
    "linux": "Tecnologia y Ciencia",
    "startup": "Tecnologia y Ciencia",
    "Criptomoneda": "Tecnologia y Ciencia",
    "bitcoin": "Tecnologia y Ciencia",
    "SectorTIC": "Tecnologia y Ciencia",
    "Meteorología": "Tecnologia y Ciencia",
    "Física": "Tecnologia y Ciencia",
    "genetica": "Tecnologia y Ciencia",
    "Neurociencia": "Tecnologia y Ciencia",
    "Debian": "Tecnologia y Ciencia",
    "retrocomp": "Tecnologia y Ciencia",
    "hacking": "Tecnologia y Ciencia",
    "Metamaterial": "Tecnologia y Ciencia",
    "Micromundo": "Tecnologia y Ciencia",
    "atomo": "Tecnologia y Ciencia",
    "Impresión3D": "Tecnologia y Ciencia",
    "Drones": "Tecnologia y Ciencia",
    "netsec": "Tecnologia y Ciencia",
    "RaspberryPi": "Tecnologia y Ciencia",
    "SysDevs": "Tecnologia y Ciencia",
    "ciencia": "Tecnologia y Ciencia",
    "astronomia": "Tecnologia y Ciencia",
    "ASTRONÁUTICA": "Tecnologia y Ciencia",
    "softlibre": "Tecnologia y Ciencia",
    "Ubuntu": "Tecnologia y Ciencia",
    "android": "Tecnologia y Ciencia",
    "smartphone": "Tecnologia y Ciencia",
    "Firefox": "Tecnologia y Ciencia",
    "Blender": "Tecnologia y Ciencia",
    "IOOcia": "Tecnologia y Ciencia",
    
    
    
    
    "salud": "Salud y Medicina",
    "medicina": "Salud y Medicina",
    "Nutrición": "Salud y Medicina",
    "sanidad": "Salud y Medicina",
    "PREVENCIÓN": "Salud y Medicina",
    "HOMEOPATÍA": "Salud y Medicina",
    "VacunaGate": "Salud y Medicina",
    "vacúname": "Salud y Medicina",
    "SANIDAD": "Salud y Medicina",
    "ebola": "Salud y Medicina",

    
    "Economía": "Negocios y Economia",
    "banca": "Negocios y Economia",
    "subvenciones": "Negocios y Economia",
    
    "ocio": "Entretenimiento y Cultura",
    "Cine": "Entretenimiento y Cultura",
    "Series": "Entretenimiento y Cultura",
    "Videojuegos": "Entretenimiento y Cultura",
    "Fotografía": "Entretenimiento y Cultura",
    "Música": "Entretenimiento y Cultura",
    "cinéfilos": "Entretenimiento y Cultura",
    "FolkloreÑ": "Entretenimiento y Cultura",
    "Cómics": "Entretenimiento y Cultura",
    "Webcomics": "Entretenimiento y Cultura",
    "ArteyMercado": "Entretenimiento y Cultura",
    "Arte": "Entretenimiento y Cultura",
    "Ilustracion": "Entretenimiento y Cultura",
    "clásica": "Entretenimiento y Cultura",
    "MUSICA70": "Entretenimiento y Cultura",
    "PALABROS": "Entretenimiento y Cultura",
    "Covers": "Entretenimiento y Cultura",
    "cultura": "Entretenimiento y Cultura",
    "Eurovision": "Entretenimiento y Cultura",
    "Poesia": "Entretenimiento y Cultura",
    "Bushido": "Entretenimiento y Cultura",
    "Arquitectura": "Entretenimiento y Cultura",
    "rol": "Entretenimiento y Cultura",
    "Podcasteros": "Entretenimiento y Cultura",
    "Viajar": "Entretenimiento y Cultura",
    "CiFi": "Entretenimiento y Cultura",
    "Gastronomia": "Entretenimiento y Cultura",
    "Clicómics": "Entretenimiento y Cultura",
    "MUSICA70":"Entretenimiento y Cultura",
    "Cocíname": "Entretenimiento y Cultura",
    "curiosidades": "Entretenimiento y Cultura",
    "acapela": "Entretenimiento y Cultura", 
    "ilustracion": "Entretenimiento y Cultura", 
    "twitter":"Entretenimiento y Cultura", 
    "reddit":"Entretenimiento y Cultura", 
    "museo": "Entretenimiento y Cultura", 
    
    
    "deportes": "Deportes",
    "Natación": "Deportes",
    "Motociclismo": "Deportes",
    "ciclismo": "Deportes",
    
    "MAmbiente": "Medioambiente y Energia",
    "Cambioclima": "Medioambiente y Energia",
    "Ecología": "Medioambiente y Energia",
    "Energias": "Medioambiente y Energia",
    "sequía": "Medioambiente y Energia",
    "Naturaleza": "Medioambiente y Energia",
    "Botánica": "Medioambiente y Energia",
    "PLANTAS": "Medioambiente y Energia",
    "AgroInfo": "Medioambiente y Energia",
    "Vivienda": "Medioambiente y Energia",
    "Animales": "Medioambiente y Energia",
    "gatos": "Medioambiente y Energia",
    "Biologia": "Medioambiente y Energia",
    "Faros": "Medioambiente y Energia",
    "mascotas": "Medioambiente y Energia",
    
    "EEUU": "Noticias Internacionales",
    "Rusia": "Noticias Internacionales",
    "Japón": "Noticias Internacionales",
    "México": "Noticias Internacionales",
    "Venezuela": "Noticias Internacionales",
    "Cataluña": "Noticias Internacionales",
    "Galiza": "Noticias Internacionales",
    "Andalucía": "Noticias Internacionales",
    "Balears": "Noticias Internacionales",
    "Canarias": "Noticias Internacionales",
    "CastillaLeón": "Noticias Internacionales",
    "deutschland": "Noticias Internacionales",
    "Siria": "Conflictos y Guerra",
    "Palestina": "Noticias Internacionales",
    "Israel": "Noticias Internacionales",
    "Africa": "Noticias Internacionales",
    "OrienteMedio": "Noticias Internacionales",
    "Cuba": "Noticias Internacionales",
    "Barcelona": "Noticias Internacionales",
    "Galicia": "Noticias Internacionales",
    "Cantabria": "Noticias Internacionales",
    "Aragón": "Noticias Internacionales",
    "Málaga": "Noticias Internacionales",
    "Roma": "Noticias Internacionales",
    "INTERNAC":"Noticias Internacionales",
    
    "Historia": "Historia y Humanidades",
    "Mitología": "Historia y Humanidades",
    "Arqueología": "Historia y Humanidades",
    "Egiptologia": "Historia y Humanidades",
    "Viñetas": "Historia y Humanidades",
    "Hallazgos": "Historia y Humanidades",
    "Asturias": "Historia y Humanidades",
    "Jurídicas": "Historia y Humanidades",
    "Etimologia": "Historia y Humanidades",
    "Psicología": "Historia y Humanidades",
    "Psicópatas": "Historia y Humanidades",
    "Filosofía": "Historia y Humanidades",
    "egiptologia": "Historia y Humanidades",
    
    
    "Humor": "Humor y Memes",
    "memes": "Humor y Memes",
    "Sarcasmos": "Humor y Memes",
    "Chorridolias": "Humor y Memes",
    "Tendenciosos": "Humor y Memes",
    "Ayuséame": "Humor y Memes",
    "podríame": "Humor y Memes",
    "Fakeame": "Humor y Memes",
    "Fakejóo": "Humor y Memes",
    "Idiocracia": "Humor y Memes",
    "meneametoday": "Humor y Memes",
    
    "GuerraCivilñ": "Conflictos y Guerra",
    "ayudaucrania": "Conflictos y Guerra",
    "Rescates": "Conflictos y Guerra",
    
    "aviacion": "Transporte",
    "aviones": "Transporte",
    "trenes": "Transporte",
    "DIRIGIBLES": "Transporte",
    "Submarino": "Transporte",
    "Motor": "Transporte",
    
    "Educación": "Educacion",
    "idiomas": "Educacion",
    "Enseñanza": "Educacion",
    "Literatura": "Educacion",
    "Libros": "Educacion",
    "lenguas": "Educacion",
    "Matemáticas": "Educacion",
    
    "Feminismo": "Cuestiones Sociales",
    "LGBT": "Cuestiones Sociales",
    "Derecho": "Cuestiones Sociales",
    "SEXOLOGÍA": "Cuestiones Sociales",
    "mujeresenlah": "Cuestiones Sociales",
    "Abuso_Animal": "Cuestiones Sociales",

    "INV": "Crimen",
    "Gamonal": "Crimen",

   
    
    "Ruinas": "Miscelaneo",
    "Relatocorto": "Miscelaneo",
    "Autorrelatos": "Miscelaneo",
    "Reportajes": "Miscelaneo",
    "Entrevistas": "Miscelaneo",
    "mnm" : "Miscelaneo",
    "Pregúntame" : "Miscelaneo",
    "microrelatos": "Miscelaneo",
    "Problemas": "Miscelaneo",
    "TuMeme": "Miscelaneo",    
    "procrastrina": "Miscelaneo",
    "Cosasguays": "Miscelaneo",
    "TuMeme": "Miscelaneo",
    "Recomiéndame": "Miscelaneo",
    "Vulcanízame": "Miscelaneo",
    "friki": "Miscelaneo",
    "Cachopos": "Miscelaneo",
    "MenteAnimal": "Miscelaneo",
    "borbones": "Miscelaneo",
    "makers": "Miscelaneo",
    "Abandonos": "Miscelaneo",
    "Marte": "Miscelaneo",
    "ACME": "Miscelaneo",
    "cervecéame": "Miscelaneo",
    "KKKantiPDMS": "Miscelaneo",
    "MUNDO_PROTE": "Miscelaneo",
    "Hazlotumismo": "Miscelaneo",
    "IMGUR": "Miscelaneo",
    "Los12monos": "Miscelaneo",
    "SeBusca": "Miscelaneo",
    "ELBAÚL": "Miscelaneo",
    "RPDC": "Miscelaneo",


    
}



In [44]:
def category_mapping(category):
    return news_category_map.get(category, "Miscelaneo")

df["category"] = df["category"].apply(category_mapping)

In [46]:
df["category"].value_counts()

category
Miscelaneo                   137983
Politica y Sociedad           94787
Entretenimiento y Cultura     39082
Tecnologia y Ciencia          14387
Medioambiente y Energia         176
Noticias Internacionales        151
Salud y Medicina                118
Historia y Humanidades          100
Transporte                       88
Cuestiones Sociales              50
Humor y Memes                    37
Educacion                        19
Negocios y Economia              17
Deportes                         13
Conflictos y Guerra               8
Crimen                            2
Name: count, dtype: int64